In [2]:
import pandas as pd
# import pygwalker as pyg
# import dtale
import re
import json

In [3]:
def _replace_df(df_: pd.DataFrame) -> pd.DataFrame:
    if type(df_) is pd.DataFrame:
        df_ = df_.rename(columns=lambda x: x.replace('\xa0', ' '))
        
        if list_of_columns := df_.select_dtypes('object').columns.to_list():
            for col in list_of_columns:
                df_[col] = df_[col].str.replace('\xa0', ' ')
    return df_

In [4]:
# Загружаем таблицу из буфера обмена
report_df = _replace_df(pd.read_clipboard(thousands=' ', decimal=','))
report_df.head(3)

,month,address,channel,category,manufacturer,brand,total_sum_money,sales_volume_in_kg
0,01.02.2023,"399851, Липецкая обл, Данковский р-н, г Данков...",TT,Хлеб,Нет производителя,Нет бренда,197740.0,NaN
1,01.02.2023,"394057, Воронежская обл, г Воронеж, Железнодор...",TT,Хлеб,Нет производителя,Нет бренда,3950.0,NaN
2,01.03.2023,"394057, Воронежская обл, г Воронеж, ул Панфило...",TT,Хлеб,Нет производителя,Нет бренда,3260.0,NaN


In [5]:
# Разбираем признак с адресом на составляющие с целью выявить самые популярные элементы
report_adress_list = report_df['address'].str.split(', ').str[1:].tolist()
report_adress_list = sum(report_adress_list, [])
report_adress_set = set(report_adress_list)
print(f'numbers elements of adress: {len(report_adress_set)}')

numbers elements of adress: 1768


In [7]:
# Сортируем по количеству повтореий и отсекаем все те что повторяются менее 2 раз

report_prefix = pd.Series(tuple(report_adress_set)).str.split().str[0].value_counts()
report_adress_prefix = report_prefix[report_prefix>1].index
report_adress_prefix


Index(['д', 'ул', 'село', 'зд', 'поселок', 'деревня', 'г', 'стр', 'влд', 'пл',
       'рп', 'тер', 'кв', 'хутор', 'помещ', 'пом', 'пр-кт', 'оф', 'уч', 'мкр',
       'пгт', 'ж/д_ст', 'пер', 'Советский', 'Ленинский', 'мгстр', 'проезд',
       'Усманский', 'Левобережный', 'слобода', 'тер.'],
      dtype='object')

In [34]:
# Составляем словарь с суфиксами
localities = {
    'region': ['обл'],
    'street': ['ул', 'пл', 'кв', 'мкр', 'проезд', 'мгстр', 'пер', 'ж/д_ст', 'р-н', 'км', 'наб', 'шоссе', 'б-р'],
    'house': ['д', 'зд', 'стр', 'влд', 'тер', 'помещ', 'пом', 'пр-кт', 'уч', 'оф', 'тер.'],
    'city': ['поселок', 'деревня', 'г', 'село', 'хутор', 'слобода', 'пгт', 'рп',],
    }


Цель составить датасет `['localities', 'suffix', 'element']`

In [37]:
loc_filter = pd.DataFrame()
worker_set = report_adress_set.copy()

for key, value in localities.items():
    for val in value:
        tmp = tuple(filter(lambda x: re.match(f'^{val}\s|.*\s{val}$', x), worker_set))
        worker_set = worker_set - set(tmp)
        tmp = list(map(lambda x: x.replace(val, '').lower().strip(), tmp))
        
        tmp_df = pd.DataFrame(tmp, columns=['element'])
        tmp_df['localities'] = key
        tmp_df['suffix'] = val
        tmp_df = tmp_df[['localities', 'suffix', 'element']]
        
        loc_filter = pd.concat((loc_filter, tmp_df), axis=0, ignore_index=True)
# other
set_len = len(worker_set)
worker_set = map(lambda x: x.lower(), worker_set)
other_list = pd.DataFrame(zip(['other']*set_len,
                            ['other']*set_len,
                            worker_set),
                            columns=['localities', 'suffix', 'element'])
loc_filter = pd.concat((loc_filter, other_list), axis=0, ignore_index=True)


loc_filter.to_csv('loc_filte.csv', index=False)
loc_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1768 entries, 0 to 1767
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   localities  1768 non-null   object
 1   suffix      1768 non-null   object
 2   element     1768 non-null   object
dtypes: object(3)
memory usage: 41.6+ KB


In [165]:
with open('adress.json', encoding='utf-8') as file:
    loc_filter = json.loads(file.read())
loc_filter.keys()

dict_keys(['region', 'city', 'street', 'house', 'other'])

In [270]:

test_df = pd.DataFrame()
for key, value in loc_filter.items():
    ad_list = sum(value.values(), [])
    tmp_df = report_df['address'].str.extract(f"({'|'.join(ad_list)})", expand=False)
    
    # Удаляем все лишние столбцы, оставляя только один
    # Преобразуем этот столбец в объект Series с помощью squeeze()
    if tmp_df.ndim>1:
        tmp_df = tmp_df.dropna(axis=1, how='all').squeeze()
    tmp_df.name = key
    
    test_df = pd.concat((test_df, tmp_df), axis=1)

test_df


,region,city,street,house,other
0,Липецкая обл,г Данков,Данковский р-н,д 34,NaN
1,Воронежская обл,г Воронеж,Железнодорожный р-н,д 1Г,NaN
2,Воронежская обл,г Воронеж,ул Панфилова,д 1Г,NaN
3,Воронежская обл,г Воронеж,Железнодорожный р-н,д 1Г,NaN
4,Воронежская обл,село Новая Усмань,Новоусманский р-н,д 26,NaN
...,...,...,...,...,...
17457,Воронежская обл,пгт Анна,Аннинский р-н,д 9,NaN
17458,Воронежская обл,г Воронеж,Центральный р-н,д 44,NaN
17459,Воронежская обл,г Воронеж,ул Новосибирская,д 35,NaN
17460,Липецкая обл,село Жерновное,Долгоруковский р-н,д 32,NaN


In [271]:
olap_df = pd.read_clipboard(decimal=',')
olap_df.head(3)

,Месяц №,Адрес,Товарная категория 3,Fact Тысруб,Fact Тн
0,1,воронеж боровое переулок_ангелиной 28,батоны,1.08252,0.01044
1,1,воронеж боровое переулок_ангелиной 28,хлеба,1.30006,0.01253
2,1,воронеж бульвар_олимпийский 14,батоны,25.21819,0.27360


In [325]:
# olap_adress_list = olap_df['Адрес'].str.split().apply(lambda x: len(x))
# olap_adress_list.value_counts()
olap_adress_set = olap_df['Адрес'].str.split()
olap_adress_set = sum(olap_adress_set, [])
olap_adress_set = set(olap_adress_set)
print(f'numbers elements of adress: {len(olap_adress_set)}')

numbers elements of adress: 2812


In [326]:
olap_suffix = pd.Series(sum(pd.Series(tuple(olap_adress_set)).str.split('_').tolist(), [])).value_counts()
olap_suffix[olap_suffix>5].head(30).index

Index(['улица', 'район', 'переулок', 'площадь', 'микрорайон', 'проезд', 'лет',
       'проспект', 'большая', 'лесная', 'победы', 'верхняя', 'область',
       'шоссе', 'революции', 'стрелковой', 'бульвар', 'красная', 'дивизии',
       'героев', 'выселки', '50', 'октября', 'новая', 'набережная', '2-я',
       'верхний'],
      dtype='object')

In [327]:
olap_suffix = ['улица', 'район', 'переулок', 'площадь', 'микрорайон', 'проезд', 'проспект', 'область', 'шоссе', 'бульвар', 'выселки', 'набережная', ]

In [ ]:
# loc_filter = dict()

# for key, value in localities.items():
#     loc_filter[key] = dict()
#     for val in value:
#         # tmp = map(lambda x: x.replace(val, '').lower().strip(), filter(lambda x: re.match(f'^{val}\s|.*\s{val}$', x), adress_set)) # удаление суффикса и перевод текста в нижний регистр
#         tmp = filter(lambda x: re.match(f'^{val}\s|.*\s{val}$', x), adress_set)
#         loc_filter[key][val] = list(tmp)

# loc_filter['other'] = {'other': list(adress_set - set(sum(sum([list(key.values()) for key in loc_filter.values()], []), [])))}

# with open('adress.json', 'w', encoding='utf-8') as file:
#     file.write(json.dumps(loc_filter, ensure_ascii=False, indent=4))

In [329]:
olap_filter = dict()

for val in olap_suffix:
    tmp = filter(lambda x: re.match(f'{val}|.*{val}', x), olap_adress_set)
    olap_filter[val] = (list(tmp))

# list(filter(lambda x: re.match(r'(улица)', x), olap_adress_list))
olap_other_set = olap_adress_set - set(sum(list(olap_filter.values()), []))
len(olap_other_set)

1681

In [330]:
olap_other_set

{'145',
 'кижеватово',
 '28б',
 'комсомолец',
 'плоское',
 'сок',
 '361',
 '46а',
 '10/74',
 'шовское',
 '100',
 'зверево',
 'ссср',
 'ищеино',
 'яблонево',
 '65',
 '241/14',
 '94а',
 'сапрон',
 'крюковка',
 '73б',
 '9а',
 '3/3',
 '42/1',
 '149',
 'революции',
 '14/1',
 'большие_избищи',
 '26',
 'пичаево',
 '54в',
 'большое-подовечье',
 '38д',
 '37',
 '62б',
 'подколодновка',
 '32а',
 'курапово',
 '110в',
 '66',
 'матвеевка',
 'территория_межрайбаз',
 'рассказово',
 '16в',
 '2/1',
 '167г',
 'завальное',
 'верхнедрезгалово',
 '313',
 '223/2',
 'красногорка',
 '113а',
 'баловнево',
 'нижнее_чесночное',
 'шемышейка',
 '44г',
 'лавы',
 'звездный',
 '33/4',
 'днт_металлург_4',
 '237',
 'березнеговатка',
 'песковатский',
 '52м',
 '107а',
 '98',
 '119в',
 '201а',
 '2л',
 'фащевка',
 '146',
 '38/3',
 'сахарный_завод',
 'студеные_хутора',
 'бутурлиновка',
 'желтые_пески',
 'новобогоявленское',
 '29/1',
 '161/1',
 'сухотиновка',
 'митрофановка',
 '115/59',
 'вешаловка',
 '10и',
 'калач',
 '22/21